# Multi-Constraint Framework for Climate Sensitivity
## Example Usage

This notebook demonstrates how to use the multi-constraint framework to estimate Equilibrium Climate Sensitivity (ECS).

In [ ]:
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import constraint modules
from src.paleoclimate import LGMConstraint, mPWPConstraint, LIGConstraint
from src.observational import HistoricalWarmingConstraint, PatternConstraint
from src.process_based import CloudFeedbackConstraint
from src.integration import MultiConstraintFramework
from src.uncertainty import UncertaintyDecomposition
from src.validation import PerfectModelTest

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Initialize Individual Constraints

In [ ]:
# Paleoclimate constraints
lgm = LGMConstraint()
mpwp = mPWPConstraint()
lig = LIGConstraint()

# Observational constraints
historical = HistoricalWarmingConstraint()

# Process-based constraints
cloud = CloudFeedbackConstraint()

print("Constraints initialized successfully!")

## 2. Calculate Individual Constraint Estimates

In [ ]:
# Calculate each constraint
lgm_constraint = lgm.calculate_constraint()
mpwp_constraint = mpwp.calculate_constraint()
lig_constraint = lig.calculate_constraint()
hist_constraint = historical.calculate_constraint()
cloud_constraint = cloud.calculate_constraint()

# Display results
constraints_list = [lgm_constraint, mpwp_constraint, lig_constraint, hist_constraint, cloud_constraint]

print("\nIndividual Constraint Estimates:")
print("=" * 60)
for c in constraints_list:
    print(f"{c['name']:15s} | Median: {c['median']:.2f} K | 90% CI: [{c['ci_90'][0]:.2f}, {c['ci_90'][1]:.2f}] K")

## 3. Visualize Individual Constraints

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ecs_values = np.linspace(1.0, 6.0, 1000)
colors = ['blue', 'green', 'orange', 'red', 'purple']

for i, c in enumerate(constraints_list):
    likelihood = c['likelihood'](ecs_values)
    ax.plot(ecs_values, likelihood, label=c['name'], color=colors[i], linewidth=2)
    ax.axvline(c['median'], color=colors[i], linestyle='--', alpha=0.5)

ax.set_xlabel('Equilibrium Climate Sensitivity (K)', fontsize=14, fontweight='bold')
ax.set_ylabel('Probability Density', fontsize=14, fontweight='bold')
ax.set_title('Individual Constraint Likelihoods', fontsize=16, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Multi-Constraint Integration

In [ ]:
# Create multi-constraint framework
mcf = MultiConstraintFramework(
    constraints=[lgm, mpwp, historical, cloud],
    prior='uniform',
    ecs_range=(1.0, 6.0),
    test_independence=True
)

# Integrate constraints
posterior = mcf.integrate_constraints(method='bayesian')

print("\nPosterior ECS Estimate:")
print("=" * 60)
print(posterior)

## 5. Plot Posterior Distribution

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
posterior.plot(ax=ax)
plt.tight_layout()
plt.show()

## 6. Independence Testing

In [ ]:
if mcf.independence_results:
    print("\nConstraint Independence Tests:")
    print("=" * 60)
    for pair, results in mcf.independence_results.items():
        print(f"\n{pair}:")
        print(f"  Correlation: {results['correlation']:.3f} (p={results['correlation_pvalue']:.3f})")
        print(f"  Mutual Information: {results['mutual_information']:.3f}")
        print(f"  Independent: {results['independent']}")
        print(f"  Interpretation: {results['interpretation']}")

## 7. Uncertainty Decomposition

In [ ]:
# Sample from posterior
posterior_samples = posterior.sample(100000)

# Decompose uncertainty
decomp = UncertaintyDecomposition()
components = decomp.decompose(posterior_samples)

print("\nUncertainty Decomposition:")
print("=" * 60)
print(f"Total Standard Deviation: {components['total_std']:.3f} K")
print(f"\nComponents:")
print(f"  Aleatory (irreducible):    {components['aleatory_std']:.3f} K ({components['aleatory_fraction']*100:.1f}%)")
print(f"  Epistemic (reducible):     {components['epistemic_std']:.3f} K ({components['epistemic_fraction']*100:.1f}%)")
print(f"  Deep uncertainty:          {components['deep_uncertainty_std']:.3f} K ({components['deep_uncertainty_fraction']*100:.1f}%)")

## 8. Sensitivity Analysis

In [ ]:
# Perform sensitivity analysis
sensitivity = mcf.sensitivity_analysis()

print("\nSensitivity Analysis:")
print("=" * 60)
for test_name, results in sensitivity.items():
    median = results['median']
    ci = results['ci_90']
    print(f"{test_name:30s} | Median: {median:.2f} K | 90% CI: [{ci[0]:.2f}, {ci[1]:.2f}] K")

## 9. Summary Statistics

In [ ]:
print("\n" + "=" * 80)
print("FINAL ECS ESTIMATE SUMMARY")
print("=" * 80)
print(f"\nMedian ECS:        {posterior.median():.2f} K")
print(f"Mean ECS:          {posterior.mean():.2f} K")
print(f"Mode ECS:          {posterior.mode():.2f} K")
print(f"\n66% CI:            [{posterior.credible_interval(0.66)[0]:.2f}, {posterior.credible_interval(0.66)[1]:.2f}] K")
print(f"90% CI:            [{posterior.credible_interval(0.90)[0]:.2f}, {posterior.credible_interval(0.90)[1]:.2f}] K")
print(f"95% CI:            [{posterior.credible_interval(0.95)[0]:.2f}, {posterior.credible_interval(0.95)[1]:.2f}] K")
print(f"\nConstraints used:  {len(mcf.constraints)}")
print(f"Prior:             {mcf.prior_type}")
print("\n" + "=" * 80)